In [1]:
import pandas
import numpy as np
import pylab as plt
import scipy
from scipy.optimize import curve_fit
import os
os.environ["PATH"] += os.pathsep + '/usr/local/texlive/2021/bin/x86_64-linux'
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt

In [2]:
plt.style.use('bmh')
plt.rc('text', usetex=True)

In [3]:
font = {'family':'serif','size':30, 'serif': ['computer modern roman']}
plt.rc('font',**font)

In [4]:
plt.rcParams['text.latex.preamble'] = r'\usepackage{amsmath}'
plt.rc('axes', grid=False, facecolor='white')

In [5]:
#We will beging by loading raw florescent data for this experiment into Python 
data_rep1 = pandas.read_excel('Raw fluorescence vs time using 4 uM PLE 1 uM Glu-C and 10 uM Opt-3 rep 1.xlsx',engine='openpyxl')
data_rep2 = pandas.read_excel('Raw fluorescence vs time using 4 uM PLE 1 uM Glu-C and 10 uM Opt-3 rep 2.xlsx',engine='openpyxl')

#Reading in the columns
data_rep1_column = data_rep1.columns
data_rep2_column = data_rep2.columns

#Importing times corresponding to fluorescence data points
T_rep1 = (data_rep1[data_rep1_column[1]].values)[1:]
T_rep1 = np.array(T_rep1,dtype='float64')

T_rep2 = (data_rep2[data_rep2_column[1]].values)[1:]
T_rep2 = np.array(T_rep2,dtype='float64')

#Importing background fluorescence values of the buffer 
buffer_rep1 = (data_rep1[data_rep1_column[-3:]].values)[1:].T
buffer_rep2 = (data_rep2[data_rep2_column[-3:]].values)[1:].T

#Defining an array with dimensions  3 x T_rep where the first dimension is 3 replicates and
#the third is fluorescent values taken every 20 seconds
F_PG_rep1 = np.zeros((3,len(T_rep1)))
F_PG_rep2 = np.zeros((3,len(T_rep2)))

In [6]:
#"PG" corresponds to esterified substrate + PLE + Glu-C, "P" to substrate + PLE, and "G" to substrate + Glu-C
#"Fmax" corresponds to fluorescence of Opt + Glu-C
#Filling the array with data
for a in range(3):
        F_PG_rep1[a,:] = (data_rep1[data_rep1_column[2+a]].values)[1:] - buffer_rep1[a,:]
        F_PG_rep2[a,:] = (data_rep2[data_rep2_column[2+a]].values)[1:] - buffer_rep2[a,:]

F_P_rep1 = np.zeros((3,len(T_rep1)))
F_P_rep2 = np.zeros((3,len(T_rep2)))

for a in range(3):
        F_P_rep1[a,:] = (data_rep1[data_rep1_column[5+a]].values)[1:] - buffer_rep1[a,:]
        F_P_rep2[a,:] = (data_rep2[data_rep2_column[5+a]].values)[1:] - buffer_rep2[a,:]
              
F_G_rep1 = np.zeros((3,len(T_rep1)))
F_G_rep2 = np.zeros((3,len(T_rep2)))
   
for a in range(3):
        F_G_rep1[a,:] = (data_rep1[data_rep1_column[8+a]].values)[1:] - buffer_rep1[a,:]
        F_G_rep2[a,:] = (data_rep2[data_rep2_column[8+a]].values)[1:] - buffer_rep2[a,:]
        
F_max_rep1 = np.zeros((3,len(T_rep1)))
F_max_rep2 = np.zeros((3,len(T_rep2)))

for a in range(3):
        F_max_rep1[a,:] = (data_rep1[data_rep1_column[11+a]].values)[1:] - buffer_rep1[a,:]
        F_max_rep2[a,:] = (data_rep2[data_rep2_column[11+a]].values)[1:] - buffer_rep2[a,:]

In [7]:
#Taking the average of times across 2 datasets      
T = (T_rep1 + T_rep2)/2 #assume no error here
T = T + 15

In [8]:
#Finding average F_PG, F_P, F_G, and F_max for plotting along with corresopnding SD

F_PG_compile = np.concatenate((F_PG_rep1,F_PG_rep2),axis=0)
F_PG_averaged = np.mean(F_PG_compile,axis=0)
std_F_PG_rep1 = np.std(F_PG_rep1, axis=0)
std_F_PG_rep2 = np.std(F_PG_rep2, axis=0)
F_PG_std = np.sqrt((std_F_PG_rep1**2 + std_F_PG_rep2**2)/4)

F_P_compile = np.concatenate((F_P_rep1,F_P_rep2),axis=0)
F_P_averaged = np.mean(F_P_compile,axis=0)
std_F_P_rep1 = np.std(F_P_rep1, axis=0)
std_F_P_rep2 = np.std(F_P_rep2, axis=0)
F_P_std = np.sqrt((std_F_P_rep1**2 + std_F_P_rep2**2)/4)

F_G_compile = np.concatenate((F_G_rep1,F_G_rep2),axis=0)
F_G_averaged = np.mean(F_G_compile,axis=0)
std_F_G_rep1 = np.std(F_G_rep1, axis=0)
std_F_G_rep2 = np.std(F_G_rep2, axis=0)
F_G_std = np.sqrt((std_F_G_rep1**2 + std_F_G_rep2**2)/4)

F_max_compile = np.concatenate((F_max_rep1,F_max_rep2),axis=0)
F_max_averaged = np.mean(F_max_compile,axis=0)
std_F_max_rep1 = np.std(F_max_rep1, axis=0)
std_F_max_rep2 = np.std(F_max_rep2, axis=0)
F_max_std = np.sqrt((std_F_max_rep1**2 + std_F_max_rep2**2)/4)

#Exporting average F_PG, F_P, F_G, and F_max for plotting along with corresopnding SD and time into Excel for plotting 
F_PG_averaged_df = pandas.DataFrame(F_PG_averaged)
F_P_averaged_df = pandas.DataFrame(F_P_averaged)
F_G_averaged_df = pandas.DataFrame(F_G_averaged)
F_max_averaged_df = pandas.DataFrame(F_max_averaged)
T_df = pandas.DataFrame(T)

F_PG_std_df = pandas.DataFrame(F_PG_std)
F_P_std_df = pandas.DataFrame(F_P_std)
F_G_std_df = pandas.DataFrame(F_G_std)
F_max_std_df = pandas.DataFrame(F_max_std)

with pandas.ExcelWriter('OUTPUT_ki_Opt-3_Fluorescence_Averaged_Progress_Curves.xlsx', engine='openpyxl') as writer:
    
    F_PG_averaged_df.to_excel(writer, sheet_name='F_PG_av')
    F_P_averaged_df.to_excel(writer, sheet_name='F_P_av')
    F_G_averaged_df.to_excel(writer, sheet_name='F_G_av')
    F_max_averaged_df.to_excel(writer, sheet_name='F_max_av')
    T_df.to_excel(writer, sheet_name='T')

    F_PG_std_df.to_excel(writer, sheet_name='F_PG_std')
    F_P_std_df.to_excel(writer, sheet_name='F_P_std')
    F_G_std_df.to_excel(writer, sheet_name='F_G_std')
    F_max_std_df.to_excel(writer, sheet_name='F_max_std')

In [9]:
#Defining substrate concentration used in the experiment
Sub_conc = [10]

#Randomly pairing F_PG, F_G, F_max, and F_P from either from rep1 (and then doing the same thing with rep2)
Pt_rep1 = np.zeros((3**4,len(T)))
Pt_rep2 = np.zeros((3**4,len(T)))
 
import itertools
index = 0
for a,b,c,d in itertools.product(range(3),repeat=4):
    Pt_rep1[index,:] = Sub_conc[0]*(F_PG_rep1[a]-F_G_rep1[b])/(F_max_rep1[c]-F_P_rep1[d])
    Pt_rep2[index,:] = Sub_conc[0]*(F_PG_rep2[a]-F_G_rep2[b])/(F_max_rep2[c]-F_P_rep2[d])
    index += 1

In [10]:
#Finding standard deviation of all product concentrations that resulted from random pairings
#Also doing the same for rep1
std_Pt_rep1 = np.std(Pt_rep1, axis=0)
std_Pt_rep2 = np.std(Pt_rep2, axis=0)

#Propagating error for finding standard deviation in the two replicates 
Pt_std = np.sqrt((std_Pt_rep1**2 + std_Pt_rep2**2)/4)
    
Pt = np.concatenate((Pt_rep1,Pt_rep2),axis=0)

#finding the average of all product concentrations that resulted from random pairings
Pt_averaged = np.mean(Pt,axis=0)

'''
#Remove commenting out for plotting
#The part below is a visual check (plotting [P]t +/- STD vs time)
plt.plot(T,Pt_averaged,'k-',linewidth=2.5)
plt.plot(T,Pt_averaged+Pt_std,'k-',linewidth=1,alpha=0.3,label='STD')
plt.plot(T,Pt_averaged-Pt_std,'k-',linewidth=1,alpha=0.3)#,label='_nolegend_')
plt.xlabel('Time (s)')
plt.ylabel('[P]t (uM)')
plt.title('[P]t with STD error vs time')
plt.legend(fontsize='x-small')
plt.show()
'''

#import sys
#sys.exit(0)

#Defining the windown of time in which kI will be fit
tmin = 150 #other tested times: #200 #300 #150 #1 #100 #150
tmax = 1500 #other tested times: #2000 #1500 #2000 #1500 #1000 #2500 #2100

# window of time that will be used for the fit
window = (T>tmin)*(T<tmax)

#Note that we checked that the obtained kI is not strongly sensitive to the chosen tmin-tmax range
print("t_min [s] =", tmin)
print("t_max [s] =", tmax)

t_min [s] = 150
t_max [s] = 1500


In [11]:
### Definining the formula [P]t = [P]inf - [I]tmin*e^(-ki*(t-tmin)) with three free fitting parameters
def func(t,Pinf,Itmin,kI): return Pinf - Itmin*np.exp(-kI*(t-tmin)) 

#Fitting the three independent parameters using curve_fit
#p0=[10,10,0.01] are the initial guesses as to what Pinf,Itmin, and kI are 10 uM, 10 uM, and 0.01 s-1, respectively
parameters = curve_fit(func,T[window],Pt_averaged[window],p0=[10,10,0.01],sigma=Pt_std[window],absolute_sigma=True)
Pinf,Itmin,kI = parameters[0]

#parameters[1] contains the covariance matrix; square root of the diagonal values are standard deviations
Pinf_SD,Itmin_SD,kI_SD = np.sqrt(np.diag(parameters[1]))

#half-life of the quinone methide intermediate
t_half = np.log(2)/kI*(1/60)

#Standard deviation error on the half-life
t_half_SD = (t_half*kI_SD)/kI

#95% confidence intervals on the errors
t_half_95_conf_int = t_half_SD*1.96
kI_95_conf_int = kI_SD*1.96
Pinf_95_conf_int = Pinf_SD*1.96
Itmin_conf_int = Itmin_SD*1.96

print("fitted k_I [1/s] =", kI)
print("fitted half-life [min] =", t_half)
print("fitted P_inf [uM] =", Pinf)
print("fitted I_tmin [uM] =", Itmin)

print("95% confidence interval on fitted k_I [1/s] =", kI_95_conf_int)
print("STD on fitted k_I [1/s] =", kI_SD)
print("95% confidence interval on fitted P_inf[uM] =", Pinf_95_conf_int)
print("95% confidence interval on fitted half-life [min] =", t_half_95_conf_int)
print("95% confidence interval on fitted I_tmin [uM] =", Itmin_conf_int)

print("time [s] in the chosen window =", T[window])
print("fitted [P]t [uM] =", func(T[window],Pinf,Itmin,kI))

'''
plt.plot(T,Pt_averaged,'k-',linewidth=2.5)
plt.plot(T,Pt_averaged+Pt_std,'k--',linewidth=1,alpha=0.3,label='STD')
plt.plot(T,Pt_averaged-Pt_std,'k--',linewidth=1,alpha=0.3)#,label='_nolegend_')
plt.plot(T[window],func(T[window],Pinf,Itmin,kI),'r--',linewidth=3, label='Exponential fit')
plt.xlabel('Time (s)')
plt.ylabel('[P]t (uM)')
plt.title('[P]t with STD error vs time')
plt.legend(fontsize='x-small')
plt.axvline(x=tmin,linestyle='-',color='black',alpha=0.3,label='_nolegend_')
plt.axvline(x=tmax,linestyle='-',color='black',alpha=0.3,label='_nolegend_')
plt.show()
'''

fitted k_I [1/s] = 0.003185705691107232
fitted half-life [min] = 3.6263403243999046
fitted P_inf [uM] = 9.108365926719447
fitted I_tmin [uM] = 6.934285741105716
95% confidence interval on fitted k_I [1/s] = 0.00011929097415892233
STD on fitted k_I [1/s] = 6.086274191781751e-05
95% confidence interval on fitted P_inf[uM] = 0.08708998971446187
95% confidence interval on fitted half-life [min] = 0.1357908456945673
95% confidence interval on fitted I_tmin [uM] = 0.08962607114969501
time [s] in the chosen window = [ 155.0715  175.081   195.0895  215.0975  235.1075  255.1175  275.1255
  295.135   315.144   335.1545  355.1635  375.1735  395.1825  415.1915
  435.2035  455.213   475.2215  495.236   515.2455  535.262   555.2705
  575.284   595.2935  615.302   635.3105  655.323   675.337   695.3465
  715.3625  735.3715  755.3815  775.3945  795.404   815.4155  835.424
  855.432   875.4445  895.4535  915.4665  935.476   955.489   975.498
  995.5065 1015.5275 1035.5365 1055.5475 1075.5585 1095.5695 

"\nplt.plot(T,Pt_averaged,'k-',linewidth=2.5)\nplt.plot(T,Pt_averaged+Pt_std,'k--',linewidth=1,alpha=0.3,label='STD')\nplt.plot(T,Pt_averaged-Pt_std,'k--',linewidth=1,alpha=0.3)#,label='_nolegend_')\nplt.plot(T[window],func(T[window],Pinf,Itmin,kI),'r--',linewidth=3, label='Exponential fit')\nplt.xlabel('Time (s)')\nplt.ylabel('[P]t (uM)')\nplt.title('[P]t with STD error vs time')\nplt.legend(fontsize='x-small')\nplt.axvline(x=tmin,linestyle='-',color='black',alpha=0.3,label='_nolegend_')\nplt.axvline(x=tmax,linestyle='-',color='black',alpha=0.3,label='_nolegend_')\nplt.show()\n"

In [12]:
#Now, knowing kI, we can also plot [I]t vs t and [I]t + [P]t vs t
#The formula for [I]t is I = dPdt/kI
#We first find dPdt

#In the manuscript, n = nr + nl
n = 4 #Spacing of n between subsequent concentrations was chosen to compute the derivative dPdt
#Note that the value of dPdt is not very sensitive to the choice of n

#dPdt = ki*[I]t
#The function above is easy to plot, but error propagation on dPdt from error on Pt_averaged is challenging
#To avoid challenging math/overestimating the error, we will take an alternative appriach:
#For each of the 6 replicates (3 technical, 2 indepednent), dPdt will be found (6 values per concentration)
#Then, the regular error propagation formula will be applied to the 6 resultant values of dPdt 

#Finding derivative curves for 81 Pt curves in each of the rep (rep 1 and rep 2)
dPdt_rep1 = (Pt_rep1[:,n:] - Pt_rep1[:,:-n])/(T[n:] - T[:-n]) #Computing dPdt for 81 Pt curves in rep1 
dPdt_rep2 = (Pt_rep2[:,n:] - Pt_rep2[:,:-n])/(T[n:] - T[:-n]) #Computing dPdt for 81 Pt curves in rep2

#Finding standard deviations corresponding to the derivative curves
std_dPdt_rep1 = np.std(dPdt_rep1, axis=0)
std_dPdt_rep2 = np.std(dPdt_rep2, axis=0)

#Propagating error for finding standard deviation in the two replicates 
dPdt_std = np.sqrt((std_dPdt_rep1**2 + std_dPdt_rep2**2)/4)
dPdt = np.concatenate((dPdt_rep1,dPdt_rep2),axis=0)

#Dinding the average of all derivative curves that resulted from product curves
dPdt_averaged = np.mean(dPdt,axis=0)

T_der = (T[n:] + T[:-n])/2 #Computing times corresponding to derivative dPdt

'''
#Remove commenting out for plotting
#The part below is a visual check(plotting [I]t with error vs time)
plt.plot(T_der,dPdt_averaged,'k-',linewidth=2.5)
plt.plot(T_der,dPdt_averaged+dPdt_std,'k-',linewidth=1,alpha=0.3,label='STD')
plt.plot(T_der,dPdt_averaged-dPdt_std,'k-',linewidth=1,alpha=0.3)#,label='_nolegend_')
plt.xlabel('Time (s)')
plt.ylabel('dPdt (uM s-1)')
plt.title('dPdt with STD error vs time')
plt.legend(fontsize='x-small')
plt.show()
'''

"\n#Remove commenting out for plotting\n#The part below is a visual check(plotting [I]t with error vs time)\nplt.plot(T_der,dPdt_averaged,'k-',linewidth=2.5)\nplt.plot(T_der,dPdt_averaged+dPdt_std,'k-',linewidth=1,alpha=0.3,label='STD')\nplt.plot(T_der,dPdt_averaged-dPdt_std,'k-',linewidth=1,alpha=0.3)#,label='_nolegend_')\nplt.xlabel('Time (s)')\nplt.ylabel('dPdt (uM s-1)')\nplt.title('dPdt with STD error vs time')\nplt.legend(fontsize='x-small')\nplt.show()\n"

In [13]:
I = dPdt_averaged/kI #Finding I 
I_std = I * np.sqrt((dPdt_std/dPdt_averaged)**2 + (kI_SD/kI)**2) #Doing error propagation on I

'''
#Remove commenting out for plotting
#The part below is a visual check(plotting [I]t with error vs time)
plt.figure(figsize=(10, 6))
plt.plot(T_der, I, label=f'10 uM of subst.')
plt.plot(T_der, I-I_std, alpha=0.3)
plt.plot(T_der, I+I_std, alpha=0.3)
plt.xlabel('Time (s)')
plt.ylabel('[I]t (uM)')
plt.title('[I]t Averaged vs T_der')
plt.grid(True)
plt.legend(fontsize='x-small')
plt.show()
'''

#The time corresponding to derivative is in the middle of time points; hence need to find Pt
#that would correspond to the time at which the derivative was found

#The errors here were computed the same way as for dPdt
P_mid_rep1 = (Pt_rep1[:,n:] + Pt_rep1[:,:-n])/2
P_mid_rep2 = (Pt_rep2[:,n:] + Pt_rep2[:,:-n])/2 

std_P_mid_rep1 = np.std(P_mid_rep1, axis=0)
std_P_mid_rep2 = np.std(P_mid_rep2, axis=0)

P_mid_std = np.sqrt((std_P_mid_rep1**2 + std_P_mid_rep2**2)/4)
P_mid = np.concatenate((P_mid_rep1,P_mid_rep2),axis=0)

P_mid_averaged = np.mean(P_mid,axis=0)

I_plus_P = I + P_mid_averaged
I_plus_P_std = np.sqrt((I_std)**2+(P_mid_std)**2)

'''
#Remove commenting out for plotting
#The part below is a visual check(plotting [I]t+[P]t with error vs time)
plt.figure(figsize=(10, 6))
plt.plot(T_der, I_plus_P, label=f'10 uM of subst.')
plt.plot(T_der, I_plus_P-I_plus_P_std, alpha=0.3)
plt.plot(T_der, I_plus_P+I_plus_P_std, alpha=0.3)
plt.xlabel('Time (s)')
plt.ylabel('[I]t + [P]t (uM)')
plt.title('[I]t+[P]t_Averaged vs T_der')
plt.grid(True)
plt.legend(fontsize='x-small')
plt.show()
'''

#Remove commenting out for plotting
#The part below is a visual check(plotting [I]t+[P]t with error vs time)
plt.figure(figsize=(10, 6))
plt.plot(T_der, I, label=f'[I]t')
plt.plot(T_der, I-I_std, alpha=0.3)
plt.plot(T_der, I+I_std, alpha=0.3)
plt.plot(T_der, I_plus_P, label=f'[I]t+[P]t')
plt.plot(T_der, I_plus_P-I_plus_P_std, alpha=0.3)
plt.plot(T_der, I_plus_P+I_plus_P_std, alpha=0.3)
plt.plot(T,Pt_averaged,'k-',linewidth=2.5, label=f'[P]t')
plt.plot(T,Pt_averaged+Pt_std,'k--',linewidth=1,alpha=0.3)
plt.plot(T,Pt_averaged-Pt_std,'k--',linewidth=1,alpha=0.3)
plt.plot(T[window],func(T[window],Pinf,Itmin,kI),'r--',linewidth=3, label='Exponential fit')
plt.legend(fontsize='x-small')
plt.axvline(x=tmin,linestyle='-',color='black',alpha=0.3,label='_nolegend_')
plt.axvline(x=tmax,linestyle='-',color='black',alpha=0.3,label='_nolegend_')
plt.xlabel('Time (s)')
plt.ylabel('Concentration (uM)')
plt.title('[I]t,[P]t, and [I]t+[P]t vs Time')
plt.grid(True)
plt.legend(fontsize='x-small')
plt.show()
    

In [14]:
#Exporting Pt_averaged and fitted Pt along with corresopnding SD and time into Excel for plotting 
Pt_averaged_df = pandas.DataFrame(Pt_averaged)
Pt_std_df = pandas.DataFrame(Pt_std)
T_df = pandas.DataFrame(T)
T_window_df = pandas.DataFrame(T[window])
Pt_fitted_df = pandas.DataFrame(func(T[window],Pinf,Itmin,kI))

with pandas.ExcelWriter('OUTPUT_ki_Opt-3_Product_Conc_Averaged_and_Fitted_Product_Progress_Curves.xlsx', engine='openpyxl') as writer:
    
    Pt_averaged_df.to_excel(writer, sheet_name='Pt_av')
    Pt_std_df.to_excel(writer, sheet_name='Pt_std')
    T_df.to_excel(writer, sheet_name='T')
    T_window_df.to_excel(writer, sheet_name='T_window')
    Pt_fitted_df.to_excel(writer, sheet_name='Pt_fitted')
    
    
#Exporting I and I_plus_P along with corresopnding SD and time into Excel for plotting 
I_df = pandas.DataFrame(I)
I_std_df = pandas.DataFrame(I_std)
I_plus_P_df = pandas.DataFrame(I_plus_P)
I_plus_P_std_df = pandas.DataFrame(I_plus_P_std)
T_der_df = pandas.DataFrame(T_der)

with pandas.ExcelWriter('OUTPUT_ki_Opt-3_I_and_I_plus_P_Progress_Curves.xlsx', engine='openpyxl') as writer:
  
    I_df.to_excel(writer, sheet_name='I')
    I_std_df.to_excel(writer, sheet_name='I_std')
    I_plus_P_df.to_excel(writer, sheet_name='I_plus_P')
    I_plus_P_std_df.to_excel(writer, sheet_name='I_plus_P_std')
    T_der_df.to_excel(writer, sheet_name='T_der')
    
#Exporting ki, half-life, P_inf, I_tmin + their corresponding error (SD or 95% confidence interval)

# Open a text file in write mode
with open('OUTPUT_kI_Experiment_All_Parameters_And_Error.txt', 'w') as f:
    # Redirect print statements to the file   
    print("fitted k_I [1/s] =", kI, file=f)
    print("fitted half-life [min] =", t_half, file=f)
    print("fitted P_inf [uM] =", Pinf, file=f)
    print("fitted I_tmin [uM] =", Itmin, file=f)
    print("95% confidence interval on fitted k_I [1/s] =", kI_95_conf_int, file=f)
    print("STD on fitted k_I [1/s] =", kI_SD, file=f)
    print("95% confidence interval on fitted P_inf[uM] =", Pinf_95_conf_int, file=f)
    print("95% confidence interval on fitted half-life [min] =", t_half_95_conf_int, file=f)
    print("95% confidence interval on fitted I_tmin [uM] =", Itmin_conf_int, file=f)